In [250]:
# !pip install azure-storage-blob
# !pip install python-dotenv

import os
from dotenv import load_dotenv
from setup_utils import fetch_data, load_data, create_time_index

CONNECTION_STRING = os.getenv("CONNECTION_STRING")

load_dotenv()
# fetch_data(CONNECTION_STRING)

(
    brand_mapping,
    macro_data,
    brand_constraint,
    pack_constraint,
    segment_constraint,
    sales_data,
    volume_variation_constraint,
) = load_data()

(
    macro_data,
    sales_data,
) = create_time_index([macro_data, sales_data])



In [297]:
import lightgbm as ltb

In [261]:
# df = sales_data[sales_data.volume.notna()].join(macro_data, on="date", how="left")
# df_test = sales_data[sales_data.volume.isna()].join(macro_data, on="date", how="left")

df = sales_data.join(macro_data, on="date", how="left")

df = df[((df.gto>0) & (df.volume>0) & (df.promotional_discount<=0) & (df.other_discounts<=0)) | df.gto.isna() ]

In [263]:
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, r2_score
import xgboost as xgb
import numpy as np
# from scipy import stats

df_og = df.copy(deep=True)

# Identify and convert categorical columns to label encoding
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


In [266]:
df.head(3)

,sku,brand,pack,size,volume,gto,promotional_discount,other_discounts,total_discounts,excise,net_revenue,maco,vilc,retail_sales_index,unemployment_rate,cpi,private_consumption,gross_domestic_saving,brad_money,gdp
date,,,,,,,,,,,,,,,,,,,,
2019-01-01,175,14,1,4,242.65,90960986.88,0.0,-598256.84,-598256.84,-27384810.42,62977919.62,47536325.82,15441593.80,92.376274,10.856350,98.286865,602773.94,125344.0,503541.2,871349.94
2019-02-01,175,14,1,4,243.38,90879757.60,0.0,-567777.63,-567777.63,-29352431.76,60959548.21,46750059.93,14209488.28,96.299010,10.856350,98.387115,602773.94,125344.0,503541.2,871349.94
2019-05-01,175,14,1,4,252.94,96540073.40,-268424.9,-472154.70,-740579.60,-26991176.48,68808317.32,57159490.21,11648827.11,103.469270,10.408203,99.380250,610350.50,125344.0,513573.5,878498.00


In [286]:
target_column = ["gto", "volume"]
drop_cols = ["sku","brand","pack","size","total_discounts","excise","net_revenue","maco","vilc"]
X = df[df.gto.notna()].drop(columns=[*target_column] + drop_cols)
y = df[df.gto.notna()][target_column]

In [277]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [278]:
scaler = StandardScaler()

In [288]:
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns,index=X.index)

In [289]:
X.head()

,promotional_discount,other_discounts,retail_sales_index,unemployment_rate,cpi,private_consumption,gross_domestic_saving,brad_money,gdp
date,,,,,,,,,
2019-01-01,0.171656,0.321766,-1.118040,-0.771336,-1.069047,-0.759173,1.672625,-1.488336,-0.403331
2019-02-01,0.171656,0.321803,-0.847766,-0.771336,-1.057192,-0.759173,1.672625,-1.488336,-0.403331
2019-05-01,0.171157,0.321921,-0.353740,-0.929305,-0.939747,-0.641834,1.672625,-1.367868,-0.294707
2019-06-01,0.169728,0.320947,-0.491551,-0.929305,-0.888096,-0.641834,1.672625,-1.367868,-0.294707
2019-07-01,0.170509,0.321118,-0.487067,-0.687970,-0.830417,-0.538759,1.672625,-1.137655,-0.225985


In [290]:
x_train = X[X.index<"2023-04-01"]
x_test = X[X.index>="2023-04-01"]

y_train = y[y.index<"2023-04-01"]
y_test = y[y.index>="2023-04-01"]

In [298]:
# X_test = df[df.gto.isna()].drop(columns=[*target_column])

# # Define a custom scorer for Weighted Absolute Percentage Error (WAPE)
# def wape_scorer(y_true, y_pred):
#     weighted_absolute_percentage_errors = np.abs((y_true - y_pred) / y_true) * np.abs(y_pred)
#     wape = np.sum(weighted_absolute_percentage_errors) / np.sum(np.abs(y_pred))
#     return np.mean(1 - wape)  # Return 1 - WAPE as a scorer

# # Create a multi-output XGBoost model
# model = MultiOutputRegressor(xgb.XGBRegressor())

# # Define the WAPE scorer using make_scorer
# wape_scorer = make_scorer(wape_scorer, greater_is_better=True)

# # Perform 5-fold cross-validation and calculate WAPE scores
# cross_val_scores = cross_val_score(model, X, y, cv=5, scoring=wape_scorer)

# # Print WAPE scores for each fold
# for fold, score in enumerate(cross_val_scores, start=1):
#     print(f'Fold {fold}: WAPE Score = {score:.4f}')

# # Print the mean WAPE score across all folds
# mean_wape_score = cross_val_scores.mean()
# print(f'Mean WAPE Score across Folds = {mean_wape_score:.4f}')

model = MultiOutputRegressor(xgb.XGBRegressor())

# model = MultiOutputRegressor(ltb.LGBMClassifier())

# Train the model on the training data
model.fit(x_train, y_train)

# Make predictions on the test data
y_pred = model.predict(x_test)

ValueError: Unknown label type: continuous. Maybe you are trying to fit a classifier, which expects discrete classes on a regression target with continuous values.

In [292]:
y_pred = pd.DataFrame(y_pred,columns = y_test.columns,index=y_test.index)

In [293]:
r2_score(y_test,y_pred)

-0.04758622096172277

In [294]:
wape = (y_test - y_pred).abs().sum() / y_test.sum()
print(wape)

gto       1.109102
volume    1.334743
dtype: float64


In [ ]:
import numpy as np
from scipy.optimize import minimize

def custom_objective(promo_discount, X, model, X_test, X_test_new):
    # Create a copy of X with the promo_discount column replaced
    X_modified = X.copy()
    X_modified[:, promo_column_index] = promo_discount

    # Predict using the modified X on the test set
    predictions_new = model.predict(X_test_new)

    # Predict using the modified X on the original test set
    predictions_original = model.predict(X_test)

    # Calculate the objective function value
    objective_value = -((predictions_new.sum() - predictions_original.sum()) / X_test.sum())

    return objective_value

# Replace these with your actual data and model
X = ...  # Your input data (including promo_discount column)
Y = ...  # Your target variables
X_test_new = ...  # New test data
X_test = ...  # Original test data
model = ...  # Your machine learning model

# Identify the column index for 'promo_discount' in X
promo_column_index = X.columns.get_loc('promo_discount')

# Initialize the initial guess for promo_discount
initial_guess = 0.1  # Replace with your initial value

# Define the optimization bounds for promo_discount (e.g., between 0 and 1)
bounds = [(0, 1)]

# Optimize promo_discount to maximize the objective function
result = minimize(custom_objective, initial_guess, args=(X.values, model, X_test.values, X_test_new.values),
                  bounds=bounds, method='L-BFGS-B')

# Extract the optimized promo_discount
optimized_promo_discount = result.x[0]

# Print the optimized promo_discount value
print(f'Optimized Promo Discount: {optimized_promo_discount:.4f}')


In [248]:
df = pd.read_excel(r"C:\Users\40107702\Documents\PyMMM\mroi-ml-lib\data\hackathon\raw\hackathon_macroeconomics_data.xlsx")

In [249]:
df.to_csv(r"C:\Users\40107702\Documents\PyMMM\mroi-ml-lib\data\hackathon\raw\hackathon_macroeconomics_data.csv",index=False)

In [255]:
df

,sku,brand,pack,size,volume,gto,promotional_discount,other_discounts,total_discounts,excise,net_revenue,maco,vilc,retail_sales_index,unemployment_rate,cpi,private_consumption,gross_domestic_saving,brad_money,gdp
0,175,14,1,4,242.65,90960986.88,0.000000e+00,-5.982568e+05,-5.982568e+05,-27384810.42,62977919.62,47536325.82,15441593.80,92.376274,10.856350,98.286865,602773.94,125344.000,503541.20,871349.94
1,175,14,1,4,243.38,90879757.60,0.000000e+00,-5.677776e+05,-5.677776e+05,-29352431.76,60959548.21,46750059.93,14209488.28,96.299010,10.856350,98.387115,602773.94,125344.000,503541.20,871349.94
2,175,14,1,4,252.94,96540073.40,-2.684249e+05,-4.721547e+05,-7.405796e+05,-26991176.48,68808317.32,57159490.21,11648827.11,103.469270,10.408203,99.380250,610350.50,125344.000,513573.50,878498.00
3,175,14,1,4,260.29,98556453.98,-1.038733e+06,-1.263461e+06,-2.302194e+06,-28407325.78,67846934.42,54858355.24,12988579.18,101.469090,10.408203,99.817024,610350.50,125344.000,513573.50,878498.00
4,175,14,1,4,262.50,98176033.39,-6.180753e+05,-1.124131e+06,-1.742206e+06,-28858235.29,67575592.04,49568230.10,18007361.94,101.534180,11.092852,100.304770,617006.06,125344.000,532745.06,883020.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6910,243,28,3,5,NaN,NaN,-1.308953e+07,-8.936375e+05,-1.398317e+07,NaN,NaN,NaN,NaN,123.707970,11.872091,130.053160,730678.25,87175.984,768195.80,982459.80
6911,243,28,3,5,NaN,NaN,-1.404901e+08,-1.887031e+08,-3.291931e+08,NaN,NaN,NaN,NaN,126.427720,12.247574,130.900010,731628.10,87175.984,769236.00,978288.20
6912,182,28,0,16,52.97,36283138.24,0.000000e+00,-9.310744e+06,-9.310744e+06,-7380086.76,19592307.36,-4914754.68,24507062.04,126.928690,11.872091,129.307080,730678.25,87175.984,768195.80,982459.80
6913,182,28,0,16,NaN,NaN,0.000000e+00,-1.897874e+07,-1.897874e+07,NaN,NaN,NaN,NaN,123.707970,11.872091,130.053160,730678.25,87175.984,768195.80,982459.80


In [254]:
x_train

,sku,promotional_discount,other_discounts,retail_sales_index,unemployment_rate,cpi,private_consumption,gross_domestic_saving,brad_money,gdp
date,,,,,,,,,,
2019-01-01,157,0.00,5.982568e+05,92.376274,10.856350,98.286865,602773.94,125344.000,503541.20,871349.94
2019-02-01,157,0.00,5.677776e+05,96.299010,10.856350,98.387115,602773.94,125344.000,503541.20,871349.94
2019-05-01,157,268424.90,4.721547e+05,103.469270,10.408203,99.380250,610350.50,125344.000,513573.50,878498.00
2019-06-01,157,1038732.88,1.263461e+06,101.469090,10.408203,99.817024,610350.50,125344.000,513573.50,878498.00
2019-07-01,157,618075.35,1.124131e+06,101.534180,11.092852,100.304770,617006.06,125344.000,532745.06,883020.25
...,...,...,...,...,...,...,...,...,...,...
2022-11-01,21,0.00,1.825560e+06,121.613050,11.117676,122.293980,740129.90,85382.875,758909.00,979321.80
2022-12-01,21,0.00,2.723728e+08,120.594570,11.117676,123.636570,740129.90,85382.875,758909.00,979321.80
2023-01-01,21,445090.76,3.025857e+08,123.191780,10.748787,125.377270,744147.30,87175.984,768181.44,993436.10
